In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
def download_page(page):
    headers = {
        #Must use this header other wise page returns 403
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0",
    }
    
    return requests.get(page, timeout=60, headers=headers).text

In [5]:
def extract_table_of_contents(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    
    resource_data = dict()
    
    resource_data["t"] = soup.find("h1").get_text().strip()
    
    author_tag = soup.find("div", {"class":"book-top-block-info-authors"})
    #Get the first child of the children iterator, convert it to string and remove the spaces
    resource_data["a"] = str(author_tag.children.__iter__().__next__()).strip()
    
    resource_data["i"] = soup.find("span", {"itemprop": "isbn"}).get_text().strip()
    
    
    
    toc_div = soup.find("div", {"id":"book-info-toc"})
    
    #toc = list()
    
    #title = soup.find("h1").get_text().strip()
    #author_tag = soup.find("div", {"class":"book-top-block-info-authors"})
    #Get the first child of the children iterator, convert it to string and remove the spaces
    #author = str(author_tag.children.__iter__().__next__()).strip()
    
    #isbn = soup.find("span", {"itemprop": "isbn"}).get_text().strip()
    
    #print(title)
    #print(author)
    #print(isbn)
    
    resource_data["c"] = list()
    #Get resource contents
    for div in toc_div.find_all("div"):
        if "book-toc-chapter-title" in div["class"]:
            #Create new section
            resource_data["c"].append(dict())
            resource_data["c"][-1]["c"] = list()
            resource_data["c"][-1]["t"] = div.get_text().strip()
            
            #print("\n" + div.get_text().strip())
            
        elif "book-toc-section-text" in div["class"]:
            resource_data["c"][-1]["c"].append({"t":div.get_text().strip()})
            #print("     " + div.get_text().strip())
        
        #print(div.name)
        #print(div["class"])
    
    #print(toc_div)
    
    return resource_data

In [6]:
def get_udemy_curriculum_by_id(courseid):
    full_page = "https://www.udemy.com/api-2.0/courses/" + courseid + "/public-curriculum-sections"
    curriculum_data = requests.get(full_page, timeout=120).json()
    
    curriculum = list()
    
    for content in curriculum_data['results']:
        content_title = content['title']
        content_subtitles = [{"t": subcont['title']} for subcont in content["items"]]
        
        curriculum.append({"t":content_title, "c":content_subtitles})
    
    return curriculum

In [7]:
import json

url = "https://www.packtpub.com/application-development/extending-microsoft-dynamics-365-operations-cookbook"

html = download_page(url)

resource_data = extract_table_of_contents(html)

#print(json.dumps(resource_data))

#download packet pub books data

In [8]:
def save_course_data(course_data):
    json_string = json.dumps(course_data)
    filename = "".join(["udemy_data/packetpub_data/",course_data['i'],".json"])
    f = open(filename, 'w')
    f.write(json_string)
    f.close()
#save_course_data(get_udemy_curriculum("machine-learning-com-weka-e-java"))

In [9]:
def download_and_save(url):
    #url = "https://www.packtpub.com/application-development/extending-microsoft-dynamics-365-operations-cookbook"

    html = download_page(url)

    resource_data = extract_table_of_contents(html)
    save_course_data(resource_data)


In [10]:
f = open("packetpub_links.txt")
url_list = f.read().split("\n")
f.close()

In [12]:
def mean(arr):
    return round(float(sum(arr))/len(arr),2)

In [18]:
import time
from ThreadPool import ThreadPool

pool = ThreadPool(20)

truncated_url_list = url_list[0:100]

done_items = 0
items_to_go = len(truncated_url_list)

elapsed_t_list = list()


def download_and_save_task(url):
    global done_items
    
    try:
        start_t = time.time()
        download_and_save("https://www.packtpub.com" + url)
        elapsed_t = time.time() - start_t
        elapsed_t = round(elapsed_t,2)
        elapsed_t_list.append(elapsed_t)
        
        done_items += 1
        print("Done with {}   {}/{}\nElapsed time: {} secs (mean: {} secs)\n"
              .format(url, done_items, items_to_go, elapsed_t, mean(elapsed_t_list)))
    except:
        done_items += 1
        print("Error with " + url)
    
pool.map(download_and_save_task, truncated_url_list)
pool.wait_completion()

print("\n------------DONE------------")


Done with /virtualization-and-cloud/aws-certified-developer-associate-tutorial-step-3-video   1/100
Elapsed time: 3.56 secs (mean: 3.56 secs)
Done with /game-development/developing-your-first-canvas-video   2/100
Elapsed time: 3.62 secs (mean: 3.59 secs)
Done with /web-development/universal-javascript-react-node-and-redux-video   3/100
Elapsed time: 3.63 secs (mean: 3.6 secs)
Done with /application-development/devops-21-toolkit-docker-swarm   4/100
Elapsed time: 3.71 secs (mean: 3.63 secs)
Done with /web-development/bootstrap-4-projects-video   5/100
Elapsed time: 3.9 secs (mean: 3.68 secs)
Done with /application-development/learning-salesforce-visual-workflow-and-process-builder-second-edition   6/100
Elapsed time: 3.92 secs (mean: 3.72 secs)
Done with /application-development/python-design-patterns-video   7/100
Elapsed time: 4.09 secs (mean: 3.78 secs)
Done with /virtualization-and-cloud/enterprise-powershell-scripting-bootcamp   8/100
Elapsed time: 4.27 secs (mean: 3.84 secs)
Done 

Done with /hardware-and-creative/hololens-beginners-guide   69/100
Elapsed time: 3.37 secs (mean: 4.24 secs)

Done with /networking-and-servers/applied-network-security   70/100
Elapsed time: 4.38 secs (mean: 4.25 secs)

Done with /application-development/mastering-opencv-3-second-edition   71/100
Elapsed time: 3.95 secs (mean: 4.24 secs)
Done with /application-development/java-9-data-structures-and-algorithms   72/100
Elapsed time: 4.09 secs (mean: 4.24 secs)
Done with /web-development/vuejs-2-cookbook   73/100
Elapsed time: 4.06 secs (mean: 4.24 secs)
Done with /application-development/introduction-go-classical-patterns-video   74/100
Elapsed time: 3.79 secs (mean: 4.23 secs)
Done with /virtualization-and-cloud/servicenow-it-operations-management   75/100
Elapsed time: 3.14 secs (mean: 4.22 secs)
Done with /networking-and-servers/jmeter-30-advanced-video   76/100
Elapsed time: 4.17 secs (mean: 4.21 secs)
Done with /big-data-and-business-intelligence/mastering-java-data-science   77/1

In [6]:
def get_info_content_links(html):
    #https://www.packtpub.com/web-development?search=&offset=&rows=100&sort=
    #class book-block-outer
    
    links = list()
    
    soup = BeautifulSoup(html, 'html.parser')
    
    for book_block_outer in soup.find_all("div", {"class": "book-block-outer"}):
        for a in book_block_outer.find_all("a"):
            href = a['href']
            if href not in links:
                links.append(href)
                
    return links
    
    
    toc_div = soup.find("div", {"id":"book-info-toc"})
    
    toc = list()
    
    h1 = soup.find("h1")
    print(h1.get_text())
    
    for div in toc_div.find_all("div"):
        if "book-toc-chapter-title" in div["class"]:
            print("\n" + div.get_text().strip())
            
        elif "book-toc-section-text" in div["class"]:
            print("     " + div.get_text().strip())
        
        #print(div.name)
        #print(div["class"])
    
    #print(toc_div)
    

In [7]:
#url = "https://www.packtpub.com/web-development?search=&offset=0&rows=10&sort="
#https://www.packtpub.com/web-development

def get_partial_links(baseurl, offset, n_results):

    params = "&".join([
        'search=',
        'offset=' + str(offset),
        'rows=' + str(n_results)
    ])

    url = baseurl + "?" + params

    html = download_page(url)

    links = get_info_content_links(html)
    
    return links

In [8]:
get_partial_links("https://www.packtpub.com/all", 1920, 5)

['/application-development/learning-nservicebus-sagas',
 '/web-development/wix-cookbook',
 '/application-development/salesforce-crm-–-definitive-admin-handbook-third-edition',
 '/application-development/openjdk-cookbook',
 '/game-development/cocos2d-game-development-blueprints']

In [126]:
url = "https://www.packtpub.com/web-development?search=&offset=11&rows=10&sort="

html = download_page(url)

links = get_info_content_links(html)
links

['/web-development/introduction-d3-video',
 '/web-development/mapping-and-styling-d3-video',
 '/web-development/learning-ionic-second-edition',
 '/web-development/d3js-4x-data-visualization-third-edition',
 '/web-development/node-craftsman-book',
 '/web-development/mastering-full-stack-react-web-development',
 '/web-development/vuejs-2-cookbook',
 '/web-development/canvas-collaboration-video',
 '/web-development/expert-data-visualization',
 '/web-development/learn-angular-2-development-building-12-apps-video']

In [164]:
all_links = list()

In [165]:
#4690 materials
#98 steps aprox
step = 48
curr_offset = 0
#n_steps = round(4700/step)
while True:
    print("Current offset: {}/{}".format(curr_offset, 4700))
    links = get_partial_links("https://www.packtpub.com/all", curr_offset, step)
    for link in links:
        all_links.append(link)
        
    if len(links) == 0:
        break
        
    curr_offset += step
    
print("Done")

Current offset: 0/4700
Current offset: 48/4700
Current offset: 96/4700
Current offset: 144/4700
Current offset: 192/4700
Current offset: 240/4700
Current offset: 288/4700
Current offset: 336/4700
Current offset: 384/4700
Current offset: 432/4700
Current offset: 480/4700
Current offset: 528/4700
Current offset: 576/4700
Current offset: 624/4700
Current offset: 672/4700
Current offset: 720/4700
Current offset: 768/4700
Current offset: 816/4700
Current offset: 864/4700
Current offset: 912/4700
Current offset: 960/4700
Current offset: 1008/4700
Current offset: 1056/4700
Current offset: 1104/4700
Current offset: 1152/4700
Current offset: 1200/4700
Current offset: 1248/4700
Current offset: 1296/4700
Current offset: 1344/4700
Current offset: 1392/4700
Current offset: 1440/4700
Current offset: 1488/4700
Current offset: 1536/4700
Current offset: 1584/4700
Current offset: 1632/4700
Current offset: 1680/4700
Current offset: 1728/4700
Current offset: 1776/4700
Current offset: 1824/4700
Current off

In [169]:
all_links[4699]

IndexError: list index out of range

In [12]:
with open('packet links.txt', 'w') as file:
    file.write("\n".join(all_links))

NameError: name 'all_links' is not defined